# CSE Path Recommendation Agent

## Overview
This notebook uses an AI-powered recommendation system to help you discover the best Computer Science Engineering (CSE) domain for your learning journey.

### What This Notebook Does:
1. **Asks you questions** about your interests, skills, and goals
2. **Analyzes your responses** using an advanced language model (LLM)
3. **Recommends a CSE domain** tailored to your profile
4. **Provides a learning roadmap** with resources and project ideas

### Prerequisites:
- An OpenRouter API key (get one free at https://openrouter.ai)
- Set the `OPENROUTER_API_KEY` environment variable on your system

Let's get started! 🚀

## Step 1: Install Dependencies
We need the `requests` library to communicate with the OpenRouter API. Run the cell below to install it.

In [ ]:
!pip install requests -q
print("✓ Dependencies installed successfully!")

## Step 2: Import Required Libraries
Let's import all the libraries we'll need for this recommendation system.

In [ ]:
import requests
import json
import os
from typing import Dict, List

print("✓ All libraries imported successfully!")

## Step 3: Load API Key from Environment

Before running this cell, make sure you've set your OpenRouter API key as an environment variable:

**Linux/Mac:**
```bash
export OPENROUTER_API_KEY="your_api_key_here"
```

**Windows:**
```cmd
set OPENROUTER_API_KEY=your_api_key_here
```

Get your free API key at: https://openrouter.ai

In [ ]:
# Load the API key from environment variables
api_key = os.getenv("OPENROUTER_API_KEY")

if not api_key:
    raise ValueError(
        "❌ OPENROUTER_API_KEY not found in environment variables.\n"
        "Please set it before running this notebook:\n"
        "  Linux/Mac: export OPENROUTER_API_KEY='your_key_here'\n"
        "  Windows: set OPENROUTER_API_KEY=your_key_here"
    )

print("✓ API Key loaded successfully!")
print(f"  API Key (first 10 chars): {api_key[:10]}...")

## Step 4: Define the Questionnaire

The questionnaire helps us understand your interests, skills, and career goals. Answer honestly for better recommendations!

In [ ]:
# Define the questionnaire with clear questions
QUESTIONS = [
    {
        "id": "background",
        "question": "What is your current background? (e.g., student, working professional, self-taught, etc.)"
    },
    {
        "id": "interests",
        "question": "What interests you most in CS? (e.g., building apps, analyzing data, cybersecurity, AI/ML, etc.)"
    },
    {
        "id": "time_commitment",
        "question": "How much time can you dedicate to learning per week? (e.g., 5 hours, 10 hours, etc.)"
    },
    {
        "id": "current_skills",
        "question": "What programming languages or technologies do you already know? (or 'none' if beginner)"
    },
    {
        "id": "career_goal",
        "question": "What is your long-term career goal? (e.g., startup founder, researcher, employee at big tech, freelancer, etc.)"
    },
    {
        "id": "learning_style",
        "question": "What's your preferred learning style? (e.g., hands-on projects, structured courses, reading docs, etc.)"
    }
]

print(f"✓ Questionnaire loaded with {len(QUESTIONS)} questions")

## Step 5: Collect User Responses

Answer each question below. Your responses will be analyzed by an AI model to provide personalized recommendations.

In [ ]:
def collect_user_responses(questions: List[Dict]) -> Dict[str, str]:
    """
    Collect user responses to the questionnaire interactively.
    
    Args:
        questions: List of question dictionaries with 'id' and 'question' keys
        
    Returns:
        Dictionary mapping question IDs to user responses
    """
    responses = {}
    
    print("\n" + "="*60)
    print("CSE PATH RECOMMENDATION QUESTIONNAIRE")
    print("="*60 + "\n")
    
    for i, q in enumerate(questions, 1):
        print(f"Question {i}/{len(questions)}")
        user_input = input(f"{q['question']}\n> ").strip()
        
        if not user_input:
            print("⚠️  Please provide a response before continuing.\n")
            # Retry if empty
            user_input = input(f"{q['question']}\n> ").strip()
        
        responses[q['id']] = user_input
        print()
    
    return responses

# Collect responses from the user
user_responses = collect_user_responses(QUESTIONS)

print("✓ Responses collected successfully!")
print("\nYour Profile Summary:")
print("-" * 40)
for q in QUESTIONS:
    print(f"{q['id'].replace('_', ' ').title()}: {user_responses[q['id']]}")
print("-" * 40)

## Step 6: Build the LLM Prompt

Now we'll create a comprehensive prompt that will guide the LLM to provide personalized recommendations based on your responses.

In [ ]:
def build_prompt(responses: Dict[str, str]) -> str:
    """
    Build a comprehensive prompt for the LLM based on user responses.
    
    Args:
        responses: Dictionary of user responses
        
    Returns:
        A formatted prompt string for the LLM
    """
    prompt = f"""You are an expert Computer Science Engineering (CSE) career advisor and mentor.

Based on the following user profile, provide a personalized CSE path recommendation:

USER PROFILE:
- Background: {responses['background']}
- Interests: {responses['interests']}
- Time Commitment: {responses['time_commitment']}
- Current Skills: {responses['current_skills']}
- Career Goal: {responses['career_goal']}
- Learning Style: {responses['learning_style']}

Please provide a comprehensive recommendation in the following format:

## RECOMMENDED CSE DOMAIN
[Specify the best-fit domain, e.g., Web Development, Machine Learning, Systems Engineering, etc.]

## REASONING
[Explain why this domain is the best fit for the user based on their profile]

## ESSENTIAL CS KNOWLEDGE TO LEARN
- [Fundamental concept 1]
- [Fundamental concept 2]
- [Fundamental concept 3]
- [Fundamental concept 4]
- [Fundamental concept 5]

## 3-6 MONTH BEGINNER ROADMAP
### Month 1-2: Foundation
- [Task 1]
- [Task 2]
- [Task 3]

### Month 2-3: Core Skills
- [Task 1]
- [Task 2]
- [Task 3]

### Month 4-6: Application & Projects
- [Project 1]
- [Project 2]
- [Project 3]

## PROJECT SUGGESTIONS (Beginner-Friendly)
1. [Project 1 with brief description]
2. [Project 2 with brief description]
3. [Project 3 with brief description]
4. [Project 4 with brief description]
5. [Project 5 with brief description]
6. [Project 6 with brief description]

## NEXT STEPS
- [Action 1]
- [Action 2]
- [Action 3]

Make your recommendations practical, actionable, and encouraging for a beginner."""
    
    return prompt

# Build the prompt
llm_prompt = build_prompt(user_responses)
print("✓ LLM prompt generated successfully!")
print(f"Prompt length: {len(llm_prompt)} characters")

## Step 7: Call OpenRouter Chat Completion API

Now we'll send your profile to a free LLM through OpenRouter to get personalized recommendations.

In [ ]:
def call_openrouter_api(api_key: str, prompt: str, model: str = "mistral-7b-instruct") -> str:
    """
    Call the OpenRouter API to get LLM recommendations.
    
    Args:
        api_key: Your OpenRouter API key
        prompt: The prompt to send to the LLM
        model: The model to use (default: mistral-7b-instruct, a free option)
        
    Returns:
        The LLM's response as a string
    """
    url = "https://openrouter.ai/api/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # Build the request payload
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "max_tokens": 2000
    }
    
    try:
        print("🔄 Sending request to OpenRouter API...")
        print(f"   Model: {model}")
        response = requests.post(url, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        
        # Parse the response
        result = response.json()
        
        # Extract the assistant's message
        if "choices" in result and len(result["choices"]) > 0:
            message = result["choices"][0]["message"]["content"]
            print("✓ Response received from OpenRouter API!")
            return message
        else:
            raise ValueError("Unexpected API response format")
            
    except requests.exceptions.RequestException as e:
        print(f"❌ API Error: {str(e)}")
        raise
    except json.JSONDecodeError as e:
        print(f"❌ JSON Parsing Error: {str(e)}")
        raise

# Call the API with your responses
print("\n" + "="*60)
print("GETTING YOUR PERSONALIZED RECOMMENDATIONS")
print("="*60)

try:
    llm_response = call_openrouter_api(api_key, llm_prompt)
    print("\n✓ Successfully generated your recommendation!\n")
except Exception as e:
    print(f"\n❌ Error occurred: {str(e)}")
    print("\nPlease make sure:")
    print("  1. Your OPENROUTER_API_KEY is valid")
    print("  2. You have internet connectivity")
    print("  3. Your API key has sufficient credits")
    raise

## Step 8: Display Formatted Results

Here are your personalized CSE path recommendations:

In [ ]:
def display_recommendations(response: str) -> None:
    """
    Display the LLM response in a clean, formatted way.
    
    Args:
        response: The LLM's recommendation response
    """
    print("\n" + "="*60)
    print("YOUR PERSONALIZED CSE PATH RECOMMENDATION")
    print("="*60 + "\n")
    print(response)
    print("\n" + "="*60)

# Display the recommendations
display_recommendations(llm_response)

## Step 9: Save Results to File

Let's save your recommendation to a file for future reference.

In [ ]:
def save_recommendation(responses: Dict[str, str], llm_response: str, filename: str = "cse_recommendation.md") -> None:
    """
    Save the user profile and LLM recommendation to a markdown file.
    
    Args:
        responses: Dictionary of user responses
        llm_response: The LLM's recommendation response
        filename: Output filename (default: cse_recommendation.md)
    """
    content = f"""# CSE Path Recommendation Report

## Your Profile
- **Background:** {responses['background']}
- **Interests:** {responses['interests']}
- **Time Commitment:** {responses['time_commitment']}
- **Current Skills:** {responses['current_skills']}
- **Career Goal:** {responses['career_goal']}
- **Learning Style:** {responses['learning_style']}

---

## AI-Powered Recommendation

{llm_response}

---

*Generated by CSE Path Recommendation Agent using OpenRouter AI*
"""
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✓ Recommendation saved to '{filename}'")
    except IOError as e:
        print(f"❌ Error saving file: {str(e)}")

# Save the recommendation
save_recommendation(user_responses, llm_response)

## Step 10: What's Next?

Congratulations! You've received your personalized CSE path recommendation. 🎉

### Recommended Next Steps:
1. **Review the Roadmap** - Examine the 3-6 month beginner roadmap and start with Month 1
2. **Choose a Learning Resource** - Pick a course platform (Coursera, Udemy, freeCodeCamp, etc.)
3. **Start with Fundamentals** - Build your foundation in CS concepts
4. **Build Projects** - Apply your learning through hands-on projects
5. **Join Communities** - Connect with others in your chosen domain (Discord, Reddit, GitHub)
6. **Stay Consistent** - Dedicate time weekly to learning and practice

### Useful Resources:
- **OpenRouter** - https://openrouter.ai (for AI assistance during learning)
- **GitHub** - https://github.com (for sharing projects and learning from others)
- **freeCodeCamp** - https://freecodecamp.org (free courses)
- **Coursera** - https://coursera.org (comprehensive CS courses)
- **Kaggle** - https://kaggle.com (if interested in ML/Data Science)

### Questions?
Feel free to re-run this notebook with different responses to explore alternative learning paths, or reach out to communities related to your chosen domain!

Good luck on your CSE journey! 🚀